The logistic.display() function in R takes in a fitted logistic regression object and outputs a linear model object in table format. When the Outbreak Investigations dataset is the dataset, this function can be used to model the probability that a person experiences a certain symptom such as nausea as a function of whether they ate a certain food such as beef curry.

The output provides the following data: 
- The crude OR(95% CI) is the unadjusted odds ratio obtained from modeling each predictor alone with the outcome.
- The adj. OR(95% CI) is the adjusted odds ratio from the multivariate model that includes all predictors simultaenously.
- P(Wald's test) is the Wald test p-value that evaluates whether the coefficient (log-odds) for that variable differs from 0.
- P(LR-test) is the likelihood ratio p value that compares the full model vs a model without that predictor.
- The output also provides model fit statistics like the log-likelihood, the number of observations, and the AIC (Akaike Information Criterion).



In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

def logistic_display(formula, data):
    """
    Mimics epiDisplay::logistic.display in R.
    Computes both crude and adjusted ORs, 95% CIs, and Wald p-values.

    Parameters:
        formula (str): example: 'nausea ~ beefcurry + saltegg + eclair + water'
        data (pd.DataFrame): dataset

    Returns:
        pd.DataFrame: Crude ORs, Adjusted ORs, p-values (Wald test), log-likelihood, number of observations, and AIC value
    """

    # Parse outcome and predictors
    outcome, rhs = formula.split("~")
    outcome = outcome.strip()
    predictors = [x.strip() for x in rhs.split("+")]

    # Fit adjusted (multivariate) model
    adj_model = smf.logit(formula=formula, data=data).fit(disp=0)
    adj_params = adj_model.params
    adj_ci = adj_model.conf_int()
    adj_pvals = adj_model.pvalues

    results = []

    for predictor in predictors:
        # --- Crude model (univariate) ---
        crude_model = smf.logit(f"{outcome} ~ {predictor}", data=data).fit(disp=0)
        crude_params = crude_model.params[predictor]
        crude_ci = np.exp(crude_model.conf_int().loc[predictor])
        crude_OR = np.exp(crude_params)

        # --- Adjusted model ---
        adj_params_pred = adj_params[predictor]
        adj_OR = np.exp(adj_params_pred)
        adj_ci_pred = np.exp(adj_ci.loc[predictor])

        # --- Wald p-values ---
        p_wald = adj_pvals[predictor]

        results.append({
            "Variable": predictor,
            "Crude OR (95% CI)": f"{crude_OR:.2f} ({crude_ci[0]:.2f}, {crude_ci[1]:.2f})",
            "Adj. OR (95% CI)": f"{adj_OR:.2f} ({adj_ci_pred[0]:.2f}, {adj_ci_pred[1]:.2f})",
            "P(Wald)": f"{p_wald:.3f}",
        })

    # Print model summary info
    print(f"\nLog-likelihood = {adj_model.llf:.4f}")
    print(f"No. of observations = {int(adj_model.nobs)}")
    print(f"AIC value = {adj_model.aic:.4f}\n")

    return pd.DataFrame(results)

#test function

#read Outbreak data
import pandas as pd
df = pd.read_csv('Outbreak.csv')

df_results = logistic_display('nausea ~ beefcurry + saltegg', df)
print(df_results)



Log-likelihood = -735.5298
No. of observations = 1094
AIC value = 1477.0596

    Variable  Crude OR (95% CI)   Adj. OR (95% CI) P(Wald)
0  beefcurry  1.03 (0.85, 1.26)  1.09 (0.71, 1.67)   0.697
1    saltegg  1.02 (0.84, 1.24)  0.94 (0.61, 1.45)   0.790
